# Setup

In [ ]:
!pip3 install opencv-python
!pip3 install matplotlib
!pip3 install numpy
!pip install opendatasets
!pip install pandas

In [ ]:
#import the library
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
import imageio
import pandas as pd

#skimage
import skimage
from skimage import color
from skimage.io import imread
from skimage import io, data, exposure
from skimage.color import rgb2gray
from skimage.feature import hog
from skimage import feature

#scipy
from scipy.cluster.vq import kmeans,vq
from scipy.stats import kurtosis
from scipy.stats import skew
from scipy.stats import entropy

#sklearn
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
folder_path = '' # substitue with your repository

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Start

In [ ]:
# Input image
data = []

def feature_extract(image, image_name):
  mean_brightness, variance_brightness, kurt_brightness, skew_brightness, entropy_brightness = brightness_extract(image)
  noise_entropy = noise_extract(image)
  snr = snr_extract(image)

  mean_red, mean_green, mean_blue, variance_red, variance_green, variance_blue, kurt_red, kurt_blue, kurt_green, skew_red, skew_blue, skew_green, avg_entropy_rgb = rgb_extract(image)
  hue_skewness, hue_kurt, hue_variance, sat_skewness, sat_kurt, sat_variance, val_skewness, val_kurt, val_variance, avg_entropy_hsv = hsv_extract(image)

  contrast, correlation, energy, homogeneity = glcm_extract(image)
  entropy_lbp, variance_lbp = lbp_extract(image)
  edgelen = edgelen_extract(image)
  hog_mean, hog_variance, hog_skewness, hog_kurt, hog_entropy = hog_extract(image)
  edgelen = edgelen_extract(image)

  features = {
      'image':image_name,
      'label': 0,
      'style': 'cubism',
      'mean_brigtness': mean_brightness,
      'variance_brightness': variance_brightness,
      'kurt_brightness': kurt_brightness,
      'skew_brightness': skew_brightness,
      'entropy_brightness': entropy_brightness,
      'noise_entropy': noise_entropy,
      'snr': snr,
      'mean_red': mean_red,
      'mean_green': mean_green,
      'mean_blue': mean_blue,
      'variance_red': variance_red,
      'variance_green': variance_green,
      'variance_blue': variance_blue,
      'kurt_red': kurt_red,
      'kurt_blue': kurt_blue,
      'kurt_green': kurt_green,
      'skew_red': skew_red,
      'skew_blue': skew_blue,
      'skew_green': skew_green,
      'avg_entropy_rgb': avg_entropy_rgb,
      'hue_skewness': hue_skewness,
      'hue_kurt': hue_kurt,
      'hue_variance': hue_variance,
      'sat_skewness': sat_skewness,
      'sat_kurt': sat_kurt,
      'sat_variance': sat_variance,
      'val_skewness': val_skewness,
      'val_kurt': val_kurt,
      'val_variance': val_variance,
      'avg_entropy_hsv': avg_entropy_hsv,
      'contrast': contrast,
      'correlation': correlation,
      'energy': energy,
      'homogeneity': homogeneity,
      'entropy_lbp': entropy_lbp,
      'variance_lbp': variance_lbp,
      'edgelen': edgelen,
      'hog_mean': hog_mean,
      'hog_variance': hog_variance,
      'hog_skewness': hog_skewness,
      'hog_kurt': hog_kurt,
      'hog_entropy': hog_entropy,
      'edgelen': edgelen
  }

  return features

height = 255
width = 255

for filename in os.listdir(folder_path):
    if filename.endswith(('.png', '.jpg', '.jpeg')):
        image_path = os.path.join(folder_path, filename)
        image = cv2.imread(image_path)
        image = cv2.resize(image, (height, width))
        features = feature_extract(image,filename)
        data.append(features)

df = pd.DataFrame(data)
df
df.to_csv('/content/drive/My Drive/df.csv', index=False)

#Feature extraction

## Brightness

In [ ]:
def brightness_extract(image):
  gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  mean_brightness = np.mean(gray_image)
  variance_brightness = np.var(gray_image)
  kurt_brightness = kurtosis(gray_image)[0]
  skew_brightness = skew(gray_image)[0]
  entropy_brightness = entropy(gray_image.ravel(), base=2)
  return mean_brightness, variance_brightness, kurt_brightness, skew_brightness, entropy_brightness

## Noise

Noise

In [ ]:
def noise_extract(image):
  gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  smoothed_image = cv2.GaussianBlur(gray_image, (5, 5), 0)
  noise = gray_image.astype(np.float32) - smoothed_image.astype(np.float32)
  noise_shifted = noise - noise.min()
  hist, _ = np.histogram(noise_shifted, bins=256, range=(0, noise_shifted.max()))
  hist_normalized = hist / hist.sum()
  entropy_noise = entropy(hist_normalized, base=2)

  return entropy_noise

Signal to Noise ratio

In [ ]:
def snr_extract(image):
  np.random.seed(42)
  noise = np.random.normal(0, 25, image.shape).astype(np.float32)
  noisy_image = image.astype(np.float32) + noise
  noisy_image = np.clip(noisy_image, 0, 255).astype(np.uint8)

  gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY).astype(np.float32)
  gray_noisy_image = cv2.cvtColor(noisy_image, cv2.COLOR_BGR2GRAY).astype(np.float32)

  signal_power= np.mean(gray_image ** 2)
  noise_power = np.mean((gray_image - gray_noisy_image) ** 2)

  if noise_power == 0:
    return float('inf')

  snr = 10 * np.log10(signal_power / noise_power)
  return snr

## Color

RGB histogram

In [ ]:
def rgb_extract(image):
  b, g, r = cv2.split(image)
  red_color = cv2.calcHist([r], [0], None, [256], [0, 256])
  green_color = cv2.calcHist([g], [0], None, [256], [0, 256])
  blue_color = cv2.calcHist([b], [0], None, [256], [0, 256])

  # Mean
  mean_red = np.mean(r)
  mean_green = np.mean(g)
  mean_blue = np.mean(b)

  # Variance
  variance_red = np.var(red_color)
  variance_blue = np.var(blue_color)
  variance_green = np.var(green_color)

  # Kurtosis
  kurt_red = kurtosis(red_color)[0]
  kurt_blue = kurtosis(blue_color)[0]
  kurt_green = kurtosis(green_color)[0]

  # Skewness
  skew_red = skew(red_color)[0]
  skew_blue = skew(blue_color)[0]
  skew_green = skew(green_color)[0]

  # Entropy
  entropy_red = entropy(red_color, base=2)
  entropy_blue = entropy(blue_color, base=2)
  entropy_green = entropy(green_color, base=2)
  avg_entropy = (entropy_red + entropy_blue + entropy_green) / 3
  avg_entropy = avg_entropy[0]

  return mean_red, mean_green, mean_blue, variance_red, variance_green, variance_blue, kurt_red, kurt_blue, kurt_green, skew_red, skew_blue, skew_green, avg_entropy

HSV histogram

In [ ]:
def hsv_extract(image):
  hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

  # Hue, Saturation, Value
  hue_hist = cv2.calcHist([hsv_image], [0], None, [180], [0, 180])
  sat_hist = cv2.calcHist([hsv_image], [1], None, [256], [0, 256])
  val_hist = cv2.calcHist([hsv_image], [2], None, [256], [0, 256])

  # Hue
  hue_mean = np.mean(hue_hist)
  hue_skewness = skew(hue_hist)[0]
  hue_kurt = kurtosis(hue_hist)[0]
  hue_variance = np.var(hue_hist)

  # Saturation
  sat_mean = np.mean(sat_hist)
  sat_skewness = skew(sat_hist)[0]
  sat_kurt = kurtosis(sat_hist)[0]
  sat_variance = np.var(sat_hist)

  # Value
  val_mean = np.mean(val_hist)
  val_skewness = skew(val_hist)[0]
  val_kurt = kurtosis(val_hist)[0]
  val_variance = np.var(val_hist)

  # Entropy
  hue_entropy = entropy(hue_hist, base=2)
  sat_entropy = entropy(sat_hist, base=2)
  val_entropy = entropy(val_hist, base=2)
  avg_entropy = (hue_entropy + sat_entropy + val_entropy) / 3
  avg_entropy = avg_entropy[0]

  return hue_skewness, hue_kurt, hue_variance, sat_skewness, sat_kurt, sat_variance, val_skewness, val_kurt, val_variance, avg_entropy

## Texture feature

Gray Level Co-Occurrence Matrix

In [ ]:
def glcm_extract(image):
  texture = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

  co_matrix = skimage.feature.graycomatrix(texture, [5], [0], levels=256, symmetric=True, normed=True)

  contrast = skimage.feature.graycoprops(co_matrix, 'contrast')[0,0]
  correlation = skimage.feature.graycoprops(co_matrix, 'correlation')[0,0]
  energy = skimage.feature.graycoprops(co_matrix, 'energy')[0,0]
  homogeneity = skimage.feature.graycoprops(co_matrix, 'homogeneity')[0,0]

  return contrast, correlation, energy, homogeneity

Local binary pattern

In [ ]:
def lbp_extract(image):
  gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  lbp = feature.local_binary_pattern(gray_image, P=8, R=1, method='uniform')
  (hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, 11), range=(0, 10))

  lbp_hist = hist.astype('float')

  variance_lbp = np.var(lbp_hist)

  lbp_hist /= lbp_hist.sum()
  entropy_lbp = entropy(lbp_hist, base=2)

  return entropy_lbp, variance_lbp

## Shape

Histogram of oriented gradients

In [ ]:
def hog_extract(image):
  gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  hog_features, hog_image = hog(gray_image, orientations=9, pixels_per_cell=(8, 8),
                          cells_per_block=(2, 2), visualize=True)
  hog_mean = np.mean(hog_features)
  hog_variance = np.var( hog_features)
  hog_skewness = skew(hog_features)
  hog_kurt = kurtosis(hog_features)
  hog_entropy = entropy(hog_features, base=2)
  return hog_mean, hog_variance, hog_skewness, hog_kurt, hog_entropy

Edge feature

In [ ]:
def edgelen_extract(image):
  gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  edges = cv2.Canny(gray_image, 100, 200)
  edgelen = np.count_nonzero(edges)

  return edgelen